<a href="https://colab.research.google.com/github/emrunali/Clinical_report_based_on_VCF_data_and_ACMG_guidelines/blob/main/ACMG_compliant_clinical_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1: Clone InterVar from its github repository**

In [1]:
!git clone https://github.com/WGLab/InterVar.git

Cloning into 'InterVar'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 697 (delta 22), reused 34 (delta 16), pack-reused 655
Receiving objects: 100% (697/697), 92.83 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (390/390), done.


**Step 2: Add mim2gene.txt and morbidmap.txt to /content/intervardb**

In [2]:
%cd InterVar

/content/InterVar


In [3]:
import sys
sys.path.append('/content/InterVar')

Make sure to modify config.ini file in the InterVar folder so that it can access annovar.

In [12]:
#Before running this cell: download annovar and upload the annovar folder with humandb
!unzip /content/annovar-20240319T032745Z-001.zip -d /content/

Archive:  /content/annovar-20240319T032745Z-001.zip
  inflating: /content/annovar/coding_change.pl  
  inflating: /content/annovar/retrieve_seq_from_fasta.pl  
  inflating: /content/annovar/.DS_Store  
  inflating: /content/annovar/table_annovar.pl  
  inflating: /content/annovar/annotate_variation.pl  
  inflating: /content/annovar/convert2annovar.pl  
  inflating: /content/annovar/example/example.simple_region  
  inflating: /content/annovar/humandb/GRCh37_MT_ensGene.txt  
  inflating: /content/annovar/example/ex1.avinput  
  inflating: /content/annovar/humandb/annovar_downdb.log  
  inflating: /content/annovar/example/example.tab_region  
  inflating: /content/annovar/humandb/hg19_example_db_generic.txt  
  inflating: /content/annovar/humandb/GRCh37_MT_ensGeneMrna.fa  
  inflating: /content/annovar/humandb/hg19_MT_ensGene.txt  
  inflating: /content/annovar/humandb/hg19_refGeneVersion.txt  
  inflating: /content/annovar/example/gene_xref.txt  
  inflating: /content/annovar/variants_

In [6]:
!apt-get install python3-vcf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2 python3-pysam
The following NEW packages will be installed:
  libhts3 libhtscodecs2 python3-pysam python3-vcf
0 upgraded, 4 newly installed, 0 to remove and 38 not upgraded.
Need to get 2,235 kB of archives.
After this operation, 7,089 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pysam amd64 0.17.0+ds-2build1 [1,749 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-vcf amd64 0.6.8+git20170215.476169c-8build1 [43.5 kB]
Fetched 2,235 kB in 3s (647 kB/s)
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 121

In [7]:
import vcf
import gzip

In [8]:
vcf_file = "/content/normal_sample.deepvariant.vcf.gz"
with gzip.open(vcf_file, 'rt') as f: #not really required
  print(f.read(100)) #read first 1200 characters in the file, just verifying

##fileformat=VCFv4.2
##DeepVariant_version=1.5.0
##FILTER=<ID=LowQual,Description="Confidence in thi


In [9]:
vcf_reader = vcf.Reader(filename = vcf_file, compressed = True)
print(vcf_reader.metadata)

OrderedDict([('fileformat', 'VCFv4.2'), ('DeepVariant_version', ['1.5.0'])])


In [ ]:
#would take longer as many variants
variant_data = [] #list of dictionaries, each dictionary has key:value pairs describing variant information from vcf file
for record in vcf_reader:
    variant = {
        'chr': record.CHROM[3:],
        'pos': record.POS,
        'ref': record.REF,
        'alt': str(record.ALT[0]),
    }
    variant_data.append(variant)

In [ ]:
print(variant_data[:10])

[{'chr': 'r', 'pos': 12807, 'ref': 'C', 'alt': 'T'}, {'chr': 'r', 'pos': 13079, 'ref': 'C', 'alt': 'G'}, {'chr': 'r', 'pos': 13418, 'ref': 'G', 'alt': 'A'}, {'chr': 'r', 'pos': 13649, 'ref': 'G', 'alt': 'C'}, {'chr': 'r', 'pos': 13813, 'ref': 'T', 'alt': 'G'}, {'chr': 'r', 'pos': 13838, 'ref': 'C', 'alt': 'T'}, {'chr': 'r', 'pos': 14248, 'ref': 'T', 'alt': 'G'}, {'chr': 'r', 'pos': 14354, 'ref': 'C', 'alt': 'A'}, {'chr': 'r', 'pos': 14522, 'ref': 'G', 'alt': 'A'}, {'chr': 'r', 'pos': 14542, 'ref': 'A', 'alt': 'G'}]


In [4]:
!gzip -dk /content/normal_sample.deepvariant.vcf.gz

In [10]:
import pysam

def filter_vcf(input_vcf_path, output_vcf_path, min_qual=20, min_vaf=0.3):
    vcf_in = pysam.VariantFile(input_vcf_path)
    vcf_out = pysam.VariantFile(output_vcf_path, 'w', header=vcf_in.header)

    for record in vcf_in.fetch():
        if record.qual < min_qual or record.filter.keys() != ['PASS']:
            continue  # Skip low-quality variants and non-PASS

        # Extract and parse VAF for each variant. This assumes VAF is the 5th value in the FORMAT field
        vaf_values = [sample.get('VAF', [0])[0] for sample in record.samples.values()]
        if all(vaf < min_vaf for vaf in vaf_values):
            continue  # Skip variants with VAF below threshold

        vcf_out.write(record)

    vcf_in.close()
    vcf_out.close()

# Example usage
filter_vcf('/content/normal_sample.deepvariant.vcf', '/content/filtered_normal_sample.deepvariant.vcf')


In [13]:
!chmod +x /content/annovar/*.pl

In [14]:
!python /content/InterVar/Intervar.py \
    --config /content/InterVar/config.ini \
    --input /content/filtered_normal_sample.deepvariant.vcf \
    --output /content/classified_variants \
    --input_type VCF \
    --buildver hg19

InterVar                                                                       
Interpretation of Pathogenic/Benign for variants using python scripts. 

.####.##....##.########.########.########..##.....##....###....########.
..##..###...##....##....##.......##.....##.##.....##...##.##...##.....##
..##..####..##....##....##.......##.....##.##.....##..##...##..##.....##
..##..##.##.##....##....######...########..##.....##.##.....##.########.
..##..##..####....##....##.......##...##....##...##..#########.##...##..
..##..##...###....##....##.......##....##....##.##...##.....##.##....##.
.####.##....##....##....########.##.....##....###....##.....##.##.....##


%prog 2.2.2 20210727
Written by Quan LI,leequan@gmail.com. 
InterVar is free for non-commercial use without warranty.
Please contact the authors for commercial use.
Copyright (C) 2016 Wang Genomic Lab

Notice: Your command of InterVar is ['/content/InterVar/Intervar.py', '--config', '/content/InterVar/config.ini', '--input', '/conte

In [15]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:00


In [17]:
import pandas as pd

In [22]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

In [35]:
import re

In [45]:
# Function to parse VCF file using pysam
def parse_vcf(vcf_path):
    vcf_data = []
    vcf_in = pysam.VariantFile(vcf_path, 'r')
    for record in vcf_in.fetch():
        qual = record.qual if record.qual else 0  # Handle None
        filter_status = ';'.join(list(record.filter.keys())) if record.filter else 'PASS'
        vcf_data.append([record.chrom, str(record.pos), record.ref, ','.join(record.alts), qual, filter_status])
    return pd.DataFrame(vcf_data, columns=['Chromosome', 'Position', 'Ref', 'Alt', 'Qual', 'Filter'])

# Function to parse .intervar file
def extract_acmg_classification(evidence_string):
    # List of known single-word ACMG classifications
    known_classifications = ['Benign', 'Pathogenic', 'Likely benign', 'Likely pathogenic', 'Uncertain significance']

    parts = evidence_string.split(';')
    intervar_part = parts[0] if parts else ""
    words = intervar_part.split()
    classification = " ".join(words[1:3]) if len(words) > 1 else "Unknown"

    # Check if the extracted classification matches any known classification (or its start)
    for known_class in known_classifications:
        if classification.startswith(known_class):
            return known_class

    return classification

# Assuming you have a function to parse the intervar file, incorporate the extraction there
def parse_intervar(intervar_path):
    intervar_data = []
    with open(intervar_path, 'r') as f:
        for line in f:
            if line.startswith('#'): continue  # Skip header lines
            parts = line.strip().split('\t')
            chr, start, ref, alt = parts[0], parts[1], parts[3], parts[4]

            # Use the corrected index for 'InterVar: InterVar and Evidence'
            intervar_evidence = parts[13]  # Adjusted based on your correction
            # Use the new function to extract ACMG classification
            acmg_classification = extract_acmg_classification(intervar_evidence)

            # Assuming inheritance pattern information is at the end (omitting the inheritance extraction for this example)
            # The logic here would remain the same as your requirements for inheritance pattern extraction

            intervar_data.append([chr, start, ref, alt, acmg_classification])

    return pd.DataFrame(intervar_data, columns=['Chromosome', 'Position', 'Ref', 'Alt', 'ACMGClassification'])


# Function to generate PDF
def generate_pdf(data, filename):
    pdf = SimpleDocTemplate(filename, pagesize=letter)
    table = Table(data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0,0), (-1,-1), 1, colors.black)
    ]))
    elems = [table]
    pdf.build(elems)

# File paths
vcf_file_path = '/content/filtered_normal_sample.deepvariant.vcf'
intervar_file_path = '/content/classified_variants.hg19_multianno.txt.intervar'

# Parse files
vcf_df = parse_vcf(vcf_file_path)
intervar_df = parse_intervar(intervar_file_path)
intervar_df['Chromosome'] = intervar_df['Chromosome'].apply(lambda x: 'chr' + x if not x.startswith('chr') else x)


In [46]:
# Assuming vcf_df is your DataFrame created from the VCF file
# and intervar_df is your DataFrame created from the .intervar file

# Slice the first 50 entries of both DataFrames
vcf_df_subset = vcf_df.iloc[:50]
intervar_df_subset = intervar_df.iloc[:50]

# Proceed to merge these subsets
merged_subset = pd.merge(vcf_df_subset, intervar_df_subset, on=["Chromosome", "Position", "Ref", "Alt"], how='inner')

# Now you can work with 'merged_subset' for your testing
print(merged_subset)

   Chromosome Position Ref Alt       Qual Filter      ACMGClassification
0        chr1    15820   G   T  22.200001   PASS                  Benign
1        chr1    69270   A   G  27.000000   PASS                  Benign
2        chr1    69511   A   G  23.200001   PASS                  Benign
3        chr1    69897   T   C  21.200001   PASS                  Benign
4        chr1   187302   A   G  20.799999   PASS  Uncertain significance
5        chr1   187302   A   G  20.799999   PASS  Uncertain significance
6        chr1   872909   A   T  26.100000   PASS  Uncertain significance
7        chr1   873542   G   A  66.900002   PASS  Uncertain significance
8        chr1   890174   A   G  30.299999   PASS  Uncertain significance
9        chr1   914618   A   G  34.200001   PASS  Uncertain significance
10       chr1   914991   G   T  20.900000   PASS           Likely benign
11       chr1   930939   G   A  65.800003   PASS  Uncertain significance
12       chr1   930939   G   A  65.800003   PASS  U

In [ ]:
# Merge DataFrames on 'Chromosome' and 'Position', ensure these columns are of matching data type
combined_df = pd.merge(vcf_df, intervar_df, on=['Chromosome', 'Position', 'Ref', 'Alt'])

# Prepare data for PDF report
data_for_pdf = [["Chromosome", "Position", "Ref", "Alt", "Qual", "Filter", "ACMG Classification", "Inheritance Pattern"]] + combined_df.values.tolist()

# Generate PDF
pdf_report_path = '/content/variant_report.pdf'
generate_pdf(data_for_pdf, pdf_report_path)

print("PDF report generated:", pdf_report_path)

In [26]:
vcf_df.head()

,Chromosome,Position,Ref,Alt,Qual,Filter
0,chr1,15820,G,T,22.200001,PASS
1,chr1,15903,G,GC,28.900000,PASS
2,chr1,69270,A,G,27.000000,PASS
3,chr1,69511,A,G,23.200001,PASS
4,chr1,69897,T,C,21.200001,PASS


In [38]:
# Adjusting chromosome names in intervar_df to match vcf_df format if needed
intervar_df.head()

,Chromosome,Position,Ref,Alt,ACMGClassification
0,chr1,15820,G,T,Benign
1,chr1,15903,-,C,Benign
2,chr1,69270,A,G,Benign
3,chr1,69511,A,G,Benign
4,chr1,69897,T,C,Benign
